In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
warnings.filterwarnings('ignore')

In [2]:
def query_from_file(file_name, params): #query database
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt): #regex
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))
    
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [3]:
# parameters
customer_id = '115'
formatted_attribute = 'Preferences'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Timing Belt"""

value='%n/a%'
customer_name='%lovelyskin%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [5]:
df = query_from_file(file_name='./query/Curated_lovelyskin.sql', params=params)
len(df)

23318

In [6]:
print(len(df[df['value'].astype(str)=='n/a']))
# df[df['value'].astype(str)=='n/a'][2500:3000]

9202


In [7]:
regex_pattern = r"(?i)essential.?oil"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]['buckets'].explode().value_counts()

150


Hair Cleaning & Treatments                               15
Hair Styling Products                                     6
Serums & Treatments                                       5
Skin Treatments                                           4
Skin Care Variety Packs                                   4
Lip Care                                                  4
Bath & Shower Additives                                   3
Face & Body Oils                                          2
Skin Care Tools & Accessories                             1
Health, Beauty, Personal Care & Hygiene Variety Packs     1
Body Washing Products                                     1
After Shave Care                                          1
Fragrances                                                1
Name: buckets, dtype: int64

In [41]:
regex_pattern = r"(?i)Vitamin.?A?B?C?D?E?"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
print("Vitamins")
df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]['buckets'].explode().value_counts()

1235
Vitamins


Serums & Treatments                                      136
Skin Care Variety Packs                                   67
Skin Treatments                                           57
Hair Cleaning & Treatments                                44
Lip Care                                                  43
Concealers & Color Correctors                             21
Hair Styling Products                                     19
Combination Makeup Products                               15
Exfoliants & Masks                                        11
Foundations & Tinted Moisturizers                          9
Eyeliners                                                  8
Mascara                                                    7
Vitamins, Minerals, & Dietary Supplements                  3
Fragrances                                                 3
Hair Products Variety Packs                                3
Combination Personal Care & Hygiene Products               2
Hair Removal Tools & Pro

In [43]:
regex_pattern = r"(?i)\bNatural.?Look"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
print("natural look")
df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]

148
natural look


attributes snake_case_name external_id  product_id  \
366    Preferences     preferences      S36414  1211872211   
867    Preferences     preferences      S36448   974553083   
869    Preferences     preferences      S36444   974553058   
870    Preferences     preferences      S36446   974553068   
871    Preferences     preferences      S36445   974553073   
957    Preferences     preferences      S36149   926666928   
1169   Preferences     preferences      S35122   847152113   
1586   Preferences     preferences      S35592   779123161   
1587   Preferences     preferences      S35596   779123272   
1588   Preferences     preferences      S35591   779123279   
1589   Preferences     preferences      S35594   779123282   
1590   Preferences     preferences      S35593   779123284   
1591   Preferences     preferences      S35595   779123285   
1605   Preferences     preferences      S35590   779124973   
1606   Preferences     preferences      S35588   779124974   
1607   Preferences     preferences      S35585   779124975   
1608   Preferences     preferences      S35586   779124977   
1609   Preferences     preferences      S35587   779124978   
1610   Preferences     preferences      S35589   779124979   
2098   Preferences     preferences      S30491   650095079   
2099   Preferences     preferences      S30492   650095081   
2100   Preferences     preferences      S30493   650095082   
2101   Preferences     preferences      S30494   650095083   
2126   Preferences     preferences      S29616   650095252   
2133   Preferences     preferences      S29615   650095263   
2134   Preferences     preferences      S29617   650095264   
2135   Preferences     preferences      S29618   650095265   
2145   Preferences     preferences        S648   650095280   
2147   Preferences     preferences        S650   650095282   
2203   Preferences     preferences      S18934   650095384   
2531   Preferences     preferences      S33438   650095970   
2845   Preferences     preferences      S18698   650096495   
2975   Preferences     preferences       S1576   650096660   
3224   Preferences     preferences      S19139   650097175   
3661   Preferences     preferences      S22416   650097958   
3731   Preferences     preferences       S2526   650098044   
3804   Preferences     preferences       S2630   650098138   
3805   Preferences     preferences       S2631   650098139   
3831   Preferences     preferences       S2634   650098167   
4415   Preferences     preferences       S3729   650099310   
4450   Preferences     preferences      S26189   650099359   
4860   Preferences     preferences      S19011   650100050   
4861   Preferences     preferences      S19012   650100051   
5188   Preferences     preferences      S19416   650100553   
5189   Preferences     preferences      S19414   650100554   
5462   Preferences     preferences      S20831   650100877   
5620   Preferences     preferences      S32471   650101065   
5650   Preferences     preferences       S4759   650101098   
5880   Preferences     preferences      S22404   650101496   
6282   Preferences     preferences      S33913   650102090   
6283   Preferences     preferences      S33917   650102091   
6284   Preferences     preferences      S33914   650102092   
6285   Preferences     preferences      S33919   650102093   
6286   Preferences     preferences      S33916   650102094   
6287   Preferences     preferences      S33915   650102095   
6288   Preferences     preferences      S33918   650102096   
7321   Preferences     preferences       S6527   650103795   
7618   Preferences     preferences       S7104   650104409   
7906   Preferences     preferences      S18237   650105125   
7965   Preferences     preferences      S29928   650105343   
8113   Preferences     preferences      S18700   650105548   
8448   Preferences     preferences      S24368   650105990   
8449   Preferences     preferences      S24365   650105991   
8858   Preferences     preferences      

In [19]:
regex_pattern = r"(?i)Formaldehyde"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]['buckets'].explode().value_counts()

16


Series([], Name: buckets, dtype: int64)

In [20]:
regex_pattern = r"(?i)Toluene"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]['buckets'].explode().value_counts()

16


Series([], Name: buckets, dtype: int64)

In [44]:
regex_pattern = r"(?i)(radical)|()"

df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]['buckets'].explode().value_counts()

547


Serums & Treatments                             90
Skin Care Variety Packs                         34
Lip Care                                        26
Skin Treatments                                 20
Concealers & Color Correctors                    9
Foundations & Tinted Moisturizers                5
Hair Cleaning & Treatments                       4
Exfoliants & Masks                               3
Hair Products Variety Packs                      2
Lotions & Moisturizers                           2
Mascara                                          2
Vitamins, Minerals, & Dietary Supplements        1
Eyeshadow                                        1
Primers                                          1
Hair Styling Products                            1
Combination Personal Care & Hygiene Products     1
Lipsticks, Glosses, & Stains                     1
Name: buckets, dtype: int64

In [30]:
regex_pattern = r"(?i)(preservatives)|()"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]['buckets'].explode().value_counts()

4


Serums & Treatments    1
Name: buckets, dtype: int64

In [29]:
regex_pattern = r"(?i)free.?radicals"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
print('free radicals')
df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]['buckets'].explode().value_counts()

346
free radicals


Serums & Treatments                             54
Skin Care Variety Packs                         26
Lip Care                                        13
Skin Treatments                                 11
Foundations & Tinted Moisturizers                5
Lotions & Moisturizers                           2
Exfoliants & Masks                               2
Hair Products Variety Packs                      2
Hair Cleaning & Treatments                       2
Vitamins, Minerals, & Dietary Supplements        1
Primers                                          1
Mascara                                          1
Combination Personal Care & Hygiene Products     1
Name: buckets, dtype: int64

In [15]:
regex_pattern = r"(?i)oil.?free"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]['buckets'].explode().value_counts()

4


Skin Treatments           1
Serums & Treatments       1
Lotions & Moisturizers    1
Name: buckets, dtype: int64

In [16]:
regex_pattern = r"(?i)paraben"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]['buckets'].explode().value_counts()
# Lipsticks, Glosses, & Stains
# Hair Products Variety Packs
# Setting Sprays & Powders

21


Lipsticks, Glosses, & Stains            4
Hair Cleaning & Treatments              3
Hair Styling Products                   3
Body Washing Products                   1
Serums & Treatments                     1
Skin Treatments                         1
Hair Products Variety Packs             1
Lotions & Moisturizers                  1
Setting Sprays & Powders                1
Facial Cleansers & Cosmetic Removers    1
Name: buckets, dtype: int64

In [24]:
regex_pattern = r"(?i)(\bscent)|()"
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]#['buckets'].explode().value_counts()

269


attributes snake_case_name external_id  product_id  \
23     Preferences     preferences      S37730  1327840068   
84     Preferences     preferences      S37224  1303561564   
167    Preferences     preferences      S37614  1266128173   
309    Preferences     preferences      S37365  1223567451   
366    Preferences     preferences      S36414  1211872211   
1207   Preferences     preferences      S36197   838834131   
1346   Preferences     preferences      S35691   806677011   
1660   Preferences     preferences         S37   650094036   
1691   Preferences     preferences      S21680   650094090   
1956   Preferences     preferences        S415   650094858   
2153   Preferences     preferences      S30902   650095290   
2539   Preferences     preferences      S21874   650095987   
2717   Preferences     preferences       S1200   650096213   
2722   Preferences     preferences       S1232   650096230   
2732   Preferences     preferences       S1221   650096273   
2844   Preferences     preferences      S17433   650096494   
2845   Preferences     preferences      S18698   650096495   
2855   Preferences     preferences      S19703   650096505   
2857   Preferences     preferences      S22162   650096507   
2858   Preferences     preferences       S1409   650096509   
2862   Preferences     preferences      S19701   650096513   
2868   Preferences     preferences      S19705   650096519   
2869   Preferences     preferences      S22163   650096520   
2870   Preferences     preferences       S1420   650096525   
2882   Preferences     preferences       S1431   650096540   
2898   Preferences     preferences      S24341   650096560   
2904   Preferences     preferences       S1446   650096568   
2906   Preferences     preferences      S18682   650096571   
2925   Preferences     preferences       S1464   650096598   
2926   Preferences     preferences      S24340   650096600   
2930   Preferences     preferences       S1469   650096605   
2932   Preferences     preferences      S18687   650096607   
2974   Preferences     preferences       S1574   650096659   
3059   Preferences     preferences      S33457   650096819   
3062   Preferences     preferences      S32853   650096822   
3080   Preferences     preferences      S18070   650096871   
3175   Preferences     preferences      S24065   650097125   
3202   Preferences     preferences      S24063   650097153   
3203   Preferences     preferences      S24066   650097154   
3228   Preferences     preferences       S1783   650097180   
3229   Preferences     preferences       S1784   650097181   
3235   Preferences     preferences       S1806   650097189   
3248   Preferences     preferences      S19659   650097202   
3249   Preferences     preferences      S19658   650097203   
3250   Preferences     preferences       S1789   650097204   
3266   Preferences     preferences       S1804   650097220   
3267   Preferences     preferences       S1805   650097221   
3268   Preferences     preferences       S1807   650097222   
3310   Preferences     preferences      S20728   650097304   
3317   Preferences     preferences      S31055   650097323   
3619   Preferences     preferences       S2140   650097880   
3654   Preferences     preferences       S2400   650097949   
3658   Preferences     preferences       S2413   650097954   
4012   Preferences     preferences      S23380   650098498   
4013   Preferences     preferences      S23378   650098499   
4021   Preferences     preferences      S23322   650098511   
4521   Preferences     preferences      S31528   650099475   
4682   Preferences     preferences       S4051   650099785   
4694   Preferences     preferences       S4019   650099802   
4726   Preferences     preferences       S4033   650099850   
4730   Preferences     preferences      S28171   650099854   
4731   Preferences     preferences      S19868   650099855   
4736   Preferences     preferences       S4035   650099860   
4738   Preferences     preferences      

In [84]:
# # Lightly
# # citrus
# # minty
# # rose
# # vanilla

# regex_pattern = r"(?i)(\bscent)|()"
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
# df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]
# # Combination Makeup Products 
# # Lip Care
# # Lipsticks, Glosses, & Stains
# # After Shave Care
# # Bath & Shower Additives
# # Health, Beauty, Personal Care & Hygiene Variety Packs

In [85]:
# regex_pattern = r"(?i)natural.?look"
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# print(len(df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]))
# df[(df['matches'].astype(str)!='[]')&(df['value'].astype(str)=='n/a')]['buckets'].explode().value_counts()
# # Combination Makeup Products
# # Lipsticks, Glosses, & Stains
# # Eyebrow Products
# # Lip Care

In [7]:
pref=df[df['value'].astype(str)=='n/a']
len(pref)

9202

In [8]:
# frizz free
# UV shield
# Oil-Free
# hydrates skin
# hydrates hair
# neutralizes free radicals
# removes impurities
# increases the production of free fatty acids
# flake-free 
# # fragrance-free
# # Free of sodium chloride
# # contains/free of  Sodium hyaluronate?
# Smudge proof/resistant
# shine-free finish--promotes shine

# acne and blemishes
# prevents water loss
# prevent clogged pores
# prevent wrinkles
# prevents future breakouts
# prevent cellular oxidation
# preventing future tissue damage
# prevent breakage

# calm skin
# absorb completely
# optimal exfoliation
# safeguard against harsh external factors
# prevent dehydration
# radiance to your complexion
# minimize signs of aging
# ultra-lubricating
# remove dirt, oil and debris
# enhance peel results
# impurities
# detoxify
# limit skin imperfections
# anti-ageing
# revitalize the skin
# tighten and firm
# brighten, retexturize, soothe irritation
# rich in amino acid

In [9]:
# regex_pattern = r"(?i)(.{0,50}$Oil.?Free.{0,50}$)|(.{0,50}$parabin.{0,50}$)|(.{0,50}$sulfate.?free.{0,50}$)|(.{50}?silicone.{50}?)|(.{50}?alternative.{50}?)"#"(amino.?acid)|(irritation)|(tighten)|(revitalize.*skin)|(anti.?ageing)|(skin.?imperfections)|(detoxify)|(peel)|(remove dirt)|(ultra-lubricating)|(aging)|(radiance)|(dehydration)|(calm skin)|(absorb)|(exfoliation)|(breakage)|(tissue damage)|(cellular oxidation)|(breakouts)|(wrinkles)|(pores)|(water loss)|(blemishes)|(acne)|(shine.?free.?finish)|(Smudge proof)|(flake.?free)|(fatty acids)|(free radicals)|(hydrates)|(Oil.?Free)|(UV shield)|(frizz free)|()"
# pref['matches'] = pref['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# x=pref[pref['matches'].astype(str)!='[]']
# print(len(x))
# y=pref[pref['matches'].astype(str)=='[]']
# # x[x['matches'].astype(str)!='[]']

In [36]:
regex_pattern = r"(?i)(.{0,50}$prevent.{0,50}$)|()"
pref['matches'] = pref['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
x=pref[pref['matches'].astype(str)!='[]']
print(len(x))
print(x['matches'][200:300])

0
Series([], Name: matches, dtype: object)


In [98]:
# regex_pattern = r"(?i)(.{50}help.{50})|()"
# pref['matches'] = pref['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
# x=pref[pref['matches'].astype(str)!='[]']
# print(len(x))
# # print(x['matches'][300:500])

In [88]:
# parameters
customer_id = '115'
formatted_attribute = 'Concern'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Timing Belt"""

value='%n/a%'
customer_name='%lovelyskin%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

df = query_from_file(file_name='./query/Curated_lovelyskin.sql', params=params)
len(df)

23372

In [89]:
new_list=[
# 'moisture balance',
'dehydration',       
'bleed',
'plaque',
# 'bacteria',
# 'gut health',
# 'touch.?up',
# 'clumps',
'redness',          
'discoloration',
'under.?eye circles', 
# 'flyaways',
# 'skin from damage',
# 'skin tissue density',
'pores',
'environmental stressors',

# 'thinning ends',
# 'excess oil',
# 'complexion',
# 'flake',
# 'brow shaping',  
# 'even sunless tan', 
# 'precision',
# 'smoothed',
# 'Dermatologist tested', 
# 'oil.?free', 
# 'color.?treated hair',
# 'residue.?free', 
# 'sulfate.?free',
# 'golden shimmer',
# 'bronze',
# 'smooth application',
         ]

In [90]:
Concern_na=df[df['value'].astype(str)=='n/a']

In [97]:
regex_pattern = r"(?i)(.{50}hydration.{50})|()"
Concern_na['matches'] = Concern_na['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
print(len(Concern_na[Concern_na['matches'].astype(str)!='[]']))
Concern_na[Concern_na['matches'].astype(str)!='[]'][0:100]

324


attributes snake_case_name external_id  product_id  \
37       Concern         concern      S37812  1321411249   
64       Concern         concern      S37611  1309948530   
76       Concern         concern      S37224  1303561564   
227      Concern         concern      S37535  1254166673   
243      Concern         concern      S37312  1236791334   
260      Concern         concern      S37311  1236791333   
298      Concern         concern      S36828  1227138973   
299      Concern         concern      S36829  1227138974   
303      Concern         concern      S36825  1227138971   
307      Concern         concern      S37361  1223567447   
312      Concern         concern      S11558  1227143637   
346      Concern         concern      S37420  1212650894   
369      Concern         concern      S37250  1202731921   
404      Concern         concern      S37247  1202731920   
405      Concern         concern      S37248  1202731922   
413      Concern         concern      S37249  1202731918   
414      Concern         concern      S37246  1202731919   
524      Concern         concern      S37009  1191933448   
534      Concern         concern      S37281  1181484339   
5593     Concern         concern      S36861  1125144479   
5626     Concern         concern      S36850  1082883246   
5627     Concern         concern      S36849  1082883249   
5643     Concern         concern      S36607  1123993443   
5726     Concern         concern      S36606   990009941   
5761     Concern         concern      S36536   956573762   
5799     Concern         concern      S36542   955049148   
5867     Concern         concern      S35682   938028431   
5954     Concern         concern      S36339   894740524   
10045    Concern         concern      S19745   650097265   
10156    Concern         concern      S35132   762555455   
10163    Concern         concern         S90   650094059   
10248    Concern         concern        S184   650094399   
10364    Concern         concern      S24950   650094675   
10451    Concern         concern      S33968   650094887   
10486    Concern         concern      S33277   650094999   
10610    Concern         concern      S33068   650095356   
10634    Concern         concern      S18376   650095420   
10943    Concern         concern      S21898   650096016   
10944    Concern         concern      S25466   650096017   
10945    Concern         concern      S21899   650096018   
10946    Concern         concern      S21897   650096019   
11059    Concern         concern      S19172   650096139   
11080    Concern         concern      S19656   650096163   
11108    Concern         concern      S24273   650096219   
11109    Concern         concern       S1210   650096258   
11132    Concern         concern       S1274   650096315   
11181    Concern         concern      S34115   650096467   
11182    Concern         concern      S34114   650096468   
11196    Concern         concern       S1409   650096509   
11205    Concern         concern      S22926   650096531   
11241    Concern         concern      S24340   650096600   
11256    Concern         concern       S1484   650096634   
11257    Concern         concern      S23526   650096637   
11265    Concern         concern       S1574   650096659   
11298    Concern         concern      S23876   650096739   
11346    Concern         concern      S32855   650096820   
11351    Concern         concern      S32881   650096850   
11478    Concern         concern      S24071   650097144   
11483    Concern         concern      S24072   650097149   
11528    Concern         concern       S1796   650097212   
11558    Concern         concern      S28285   650097259   
11617    Concern         concern       S1890   650097439   
11713    Concern         concern      S17495   650097679   
11766    Concern         concern       S2052   650097752   
11767    Concern         concern       S2053   650097753   
11974    Concern         concern       S2802  

In [ ]:

S37247: Featuring a soft, creamy texture, Laura Geller Smart Pout Transfer-Proof Matte Lipstick offers hydration that lasts all day with transfer-proof color.

In [ ]:
Lipstick
S21525: Curved grip pen gives total control so color applies evenly without bleeding or skipping.
S14022: It can help stop bleeding in the event of a minor surface nick or cut and preps the skin for an electric razor shave.
S25504: This new full-coverage lipstick formula provides the one-swipe color that made Moxie Lipstick a favorite, with a new modern finish, a primer-infused comfortable feel, and long lasting color that resists feathering and bleeding

In [ ]:
Plaque-Maybe for Tools
S22740: In addition to breaking up plaque, preventing tartar buildup and lifting stains, the gentle bristles work to massage the tongue and gums, ensuring you never over-brush. 

In [ ]:
Clumps:hair
S30704 : The high-performance, clump- and flake-free formula features a specially-designed brush with a ball tip at the end to reach small, fine lashes as well as volumizing bristles to distribute product evenly and create maximum volume.

In [ ]:
Flyway-Use for hair
S1083: The advanced formula protects against humidity and locks in moisturize to prevent flyaways.

In [ ]:
Environmental Protection
S31335: TiZO Photoceutical Environmental Skin Protectant soothes and protects from environmental pollutants. 
S16387: Rich in ingredients renowned for their restructuring, nourishing and hydrating properties, it helps preserve a youthful appearance for hands, and ensures their protection against harsh environmental factors 

In [ ]:
Thin Ends-Use for hair
S23736: This synergystic system works together to prevent or slow thinning and loss of hair while soothing scalp itching and irritation.

In [11]:
# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# list_values

In [13]:
# kwp=KeywordProcessor()
# kwp.add_keywords_from_list(list_values)
# # Extract Key Words
# Concern_na['matches']=Concern_na['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #
# Concern_na.head(1)

In [14]:
# kwp.add_keywords_from_list(new_list)
# # Extract Key Words
# Concern_na['new_matches']=Concern_na['long_desc'].apply(lambda x: kwp.extract_keywords(x)) 
# new=Concern_na[Concern_na['new_matches'].astype(str)=='[]']

In [15]:
# print(len(new))
# print(new['buckets'].explode().value_counts()[0:10])

In [16]:
# Concern_na[Concern_na['new_matches'].astype(str)!='[]']['new_matches'].explode().value_counts()

In [17]:
# new[2000:2500]

In [ ]:
Concern_na_matches=Concern_na[Concern_na['matches'].astype(str)!='[]']
Concern_na_matches['matches']=Concern_na_matches['matches'].apply(lambda x: sorted(x, key=natural_key))
Concern_na_matches['matches']

In [ ]:
Concern_na_matches['matches']=[list(set(sublist)) for sublist in Concern_na_matches['matches']]
Concern_na_matches['matches']

In [ ]:
Concern_na['matches'].explode().value_counts()

In [ ]:
x

In [ ]:
x[x['value'].astype(str)!='Greater than 30 Inches'][100:150]

In [ ]:
df[df['external_id'].astype(str)=='00739441']

In [ ]:
# x['matches'].explode().value_counts()

In [ ]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
    # data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [ ]:
split_col(x)

In [ ]:
len(x['matches'].explode().value_counts())

In [ ]:
# error

In [ ]:
x['matches']=x['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\'",'',str(measure)))
x

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
#         numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
x['matches']=x['matches'].apply(lambda x: remove_fractions(x))
x

In [ ]:
x['matches']=x['matches'].astype(float)

In [ ]:
x

# matches

In [ ]:
matches=x
matches

In [ ]:
matches['matches']=matches['matches'].astype(float)

In [ ]:
# range function builder

string = range_string.replace(metric, '')
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return "{original_val}{metric}"
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)

In [ ]:
matches[matches['value'].astype(str)=='Greater than 50 Inches']

In [ ]:
matches['matches']=matches['matches'].astype(float)
matches[curation_col] = matches['matches'].apply(range_app)
matches

In [ ]:
matches['Q:outside_diameter'].explode().value_counts()

# fixed_na

In [ ]:
fixed_na=matches[matches['Q:outside_diameter'].astype(str)=='n/a']

In [ ]:
fixed_na['matches']=fixed_na['matches'].astype(float).apply(lambda x: x-1) #have to add 1 because these were 100.90 and it could not classify it
# na

In [ ]:
fixed_na[curation_col] = fixed_na['matches'].apply(range_app)
fixed_na

In [ ]:
fixed_na['Q:outside_diameter'].explode().value_counts()

# Wipe

In [ ]:
wipe=df[['external_id']].astype(str)
wipe['Q:outside_diameter']=''
wipe

# Null Values

In [ ]:
na_external_id=df[df['matches'].astype(str)=='[]']
na_external_id['external_id']=na_external_id['external_id'].astype(str)

In [ ]:
na_external_id

In [ ]:
error

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)

In [ ]:
looks_good('Motion Industries', attribute, df, wipe)

In [99]:
import pandas as pd
import numpy as np
from collections import defaultdict
from fuzzywuzzy import fuzz
from sklearn.cluster import AffinityPropagation
import os
import groupby_toolz
import spacy #https://spacy.io/usage/linguistic-features
# install spacy here https://spacy.io/usage
# import en_core_web_sm
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from IPython.display import display
pd.options.display.max_columns = None
from decimal import *
TWOPLACES = Decimal(10) ** -2
# https://stackoverflow.com/questions/21903842/how-do-i-compare-two-strings-in-python

#Colors
import webcolors
import math

In [100]:
#######################################################################  Syndication Data  ##########################################################################
def col_group(df, fuzzy = 'partial',return_dict = False):
    '''
    Affinity Clustering to sort column names into clusters based on various fuzzy match
    param:
    =======
    df: Pandas DataFrame, syndicated DataFrame
    fuzzy: str, default value is partial, to select fuzzy ratio type: ['ratio','partial','token_sort','token_set','partial_sort','partial_set'] 
    return_dict: bool, if False, prints output. If True, returns a dict with each cluster. Default value False 
    '''
    # Modified from: https://bit.ly/3vRuE1y
    assert isinstance(return_dict, bool), "return_dict argument must be boolean"
    assert fuzzy in ['ratio','partial','token_sort','token_set','partial_sort','partial_set'],\
    "fuzzy must be in 'ratio','partial','token_sort','token_set','partial_sort', or 'partial_set'"
    words = np.asarray(df.columns)
    # Pick a fuzzy ratio
    if fuzzy == 'ratio':
        lev_similarity = np.array([[fuzz.ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'partial':
        lev_similarity = np.array([[fuzz.partial_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'token_sort':
        lev_similarity = np.array([[fuzz.token_sort_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'token_set':
        lev_similarity = np.array([[fuzz.token_set_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'partial_sort':
        lev_similarity = np.array([[fuzz.partial_token_sort_ratio(w1,w2) for w1 in words] for w2 in words])
    elif fuzzy == 'partial_set':
        lev_similarity = np.array([[fuzz.partial_token_set_ratio(w1,w2) for w1 in words] for w2 in words])
    # Affinity clustering
    affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
    affprop.fit(lev_similarity)
    # Simply print output
    if return_dict == False:
        for cluster_id in np.unique(affprop.labels_):
            exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
            cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
            cluster_str = ", ".join(cluster)
            print(" - *%s:* %s" % (exemplar, cluster_str))
            print('\b')
    # Returns a dictionary instead
    elif return_dict == True:
        dict_ = defaultdict()
        for cluster_id in np.unique(affprop.labels_):
            exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
            cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
            cluster_str = ", ".join(cluster)
            dict_[exemplar]=[f'{(exemplar, cluster_str)}']
        return dict_
# Print out values from two columns that have uncommon values
def print_att(df,one,two):
    for i in range(len(df)):
        Str1 = str(df[f'{one}'][i])
        Str2 = str(df[f'{two}'][i])
        Partial_Ratio = fuzz.token_set_ratio(Str1,Str2) 
        if Partial_Ratio<100:
            print(f"{str(df[f'{one}'][i])}, {str(df[f'{two}'][i])}")
            
            
def mult_func_two(df,one,two):
    new=pd.DataFrame()
    new['package_new'] = df[[    f'{one}',    f'{two}']].apply(lambda x: ','.join(x.dropna()), axis=1)
    print(len(new['package_new'].explode().value_counts()))
    print(new['package_new'].explode().value_counts()[0:500])
def mult_func_three(df,one,two,three):
    new=pd.DataFrame()
    new['package_new'] = df[[    f'{one}',    f'{two}',    f'{three}']].apply(lambda x: ','.join(x.dropna()), axis=1)
    print(len(new['package_new'].explode().value_counts()))
    print(new['package_new'].explode().value_counts()[0:500])
def mult_func_four(df,one,two,three,four):
    new=pd.DataFrame()
    new['package_new'] = df[[    f'{one}',    f'{two}',    f'{three}',f'{four}']].apply(lambda x: ','.join(x.dropna()), axis=1)
    print(len(new['package_new'].explode().value_counts()))
    print(new['package_new'].explode().value_counts()[0:500])



#######################################################################    Color    ##########################################################################
colorSplitRegex = r'-| |/|\\|,'
# G E T   A   C O L O R   N A M E
# get the closest color from an rgb tuple
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]
# get the color name either from webcolors or closest_color
def get_colour_name(requested_colour):
    try:
        closest_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
    return closest_name.title()
# G E T   A   C O L O R   V A L U E
# convert a string to an rgb tuple
def parseColor(string):
    try:
        # convert the color string to rgb, default spec is css3
        return webcolors.name_to_rgb(string)
    except Exception:
        pass
# parse raw data for strings we can match on
# if the string has a dash, slash, or space try each color individually
# and average out the RGB values with a simple a+b+c/n forumla
def parseColorData(data):
    # init an empty array for color values
    colorValues = []
    # split it on dashes, spaces, slashes, or commas
    if re.split(colorSplitRegex, data):
        for color in re.split(colorSplitRegex, data):
            colorValues.append(parseColor(color))
    # if it can't be split just full send it
    else:
        colorValues.append(parseColor(color))
    # remove None values
    colorValues = [
        colorValue for colorValue in colorValues if colorValue is not None]
    if not len(colorValues):
        return data
    # find the average color and convert it to a color name
    #######################################################
    # init an average color value
    averageColor = [0, 0, 0]
    for colorValue in colorValues:
        for index, value in enumerate(colorValue):
            averageColor[index] += value
    averageColor = [math.floor(value / (len(colorValues) or 1))
                    for value in averageColor]
    # just change this to a return to run this function
    return get_colour_name(averageColor)





#######################################################################    Clean/Merge Data    ##########################################################################

# Finds the first digit (basically skips over words and letters to find the first number)
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

# i finds the first digit: if it equals zero then there is not a digit, if it equals the end then leave it
def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

            
# NATURAL SORT
######################################################################
# sort the number naturally so 5 will come before 15
def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))
# x['matches']= x['matches'].apply(lambda x: sorted(x, key=natural_key))



#------------atof and natural_keys are complementary-------------
# atof and natural_keys: Will naturally sort 5 infront of 15
def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]
#########################################################################





# REPLACE
#########################################################################
# # Replace specific text in a string: Exmapel1  to  Example1
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# replacement={'Exmapel1':'Example1',    }
# df= df.apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)


def replace_all(text, dic):
    '''
    Given a text and replacement dictionary, performs multiple str.replace()
    Arguments:
    ==========
    text: str, text to perform str.replace()
    dic: dict, where each value is the string to replace, and each key is the string to replace with
    '''
    for key, value in dic.items():
        for v in value:
            text = text.replace(v, key)
    return text
# power['min_temp_sj']=power['min_temp_sj'].apply(lambda x: replace_all(x,{'':['Ã‚Â°F']}))


# Replace all letters and some special characters with a blank
# data['max']=data['max'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(measure)))
#########################################################################




# FRACTIONS
#########################################################################
# Convert a fraction that does not have an int in front of the fraction "1/3"
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
     
    
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))
#########################################################################





# CONVERTING FRACTIONS, WHOLE NUMBERS, AND NUMBERS WITH LETTERS
#########################################################################
#Converts the number to a decimal if it has a decimal number but leaves off .0 if it is an int. Be sure to conver it back to a string and add -"in"
def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()
# from decimal import *
# TWOPLACES = Decimal(10) ** -2
# tubes_new['height_new']=tubes_new['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))



 
# converts the value to a float if possible and skips the value if not   
def convert_float(val):
    try:
        return float(val)
    except:
        return str(val)
    
    
    

# Definition to round a number
def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x   
    
# Excel might convert 8-1/2 to 08/01/2002 and we need to conver this back into 8.5  
def date_to_decimal(numbers):
    if numbers.find('/') > -1:
        try:
            numbers=str(numbers)
            one, two,three = numbers.split('/')
            tmp = float(two)/float(three[-1])
            num = int(one) + tmp
            return str(num)
        except:
            return numbers
    else:
        return numbers

# Convert a fraction "1/4" that is written as a date "1-4-2021"
def remove_basic_date(numbers):
    if numbers.find('-2021') > -1:
        first, second, year = numbers.split('-')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
    

# Convert a fraction "1/4" that is written as a date "2021-01-04"
def remove_date_year_first(numbers):
    if numbers.find('2021-') > -1:
        year, first, second = numbers.split('-')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
# Convert a fraction "1/4" that is written as a date "1/4/2021"    
def remove_date_year_last(numbers):
    if numbers.find('/2021') > -1:
        first, second,year = numbers.split('/')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
    
regex_year_month_day = r"(?i)(\d\d\d\d\s\d\d\s\d\d)|()"   

def re_extract(pattern, txt): #regex
    try:
        matches = re.findall(pattern, txt)
        tmp_matches = []
        for match in matches:
            for tup in match:
                year, first, second = tup.split(' ')
                num=float(first)+float(second)/(float(year[2:4]))

                return str(num)
                if tup != '':
                    tmp_matches.append(tup)
        return list(set(tmp_matches))
    except:
        return txt



# Just merging everything into one function
def final_done(df,col):
    return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)|#','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda x: remove_basic_date(x)).apply(lambda x: remove_date_year_first(x)).apply(lambda x: remove_date_year_last(x)).apply(lambda measure: re.sub(r'(?<=\d)\s?-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: re_extract(regex_year_month_day,x)).apply(lambda x: round_string_float(x))
#     return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda measure: re.sub(r'\-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: round_string_float(x))
    
def final_done(df,col):
    return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)|#','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda x: remove_basic_date(x)).apply(lambda x: remove_date_year_first(x)).apply(lambda x: remove_date_year_last(x)).apply(lambda measure: re.sub(r'(?<=\d)\s?-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: re_extract(regex_year_month_day,x)).apply(lambda x: round_string_float(x))
#     return df[col].apply(lambda measure: re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)','',str(measure))).apply(lambda measure: re.sub(r'00:00:00|00:00|0:00','',str(measure))).apply(lambda measure: measure.rstrip()).apply(lambda measure: re.sub(r'\-',' ',str(measure))).apply(lambda x: date_to_decimal(x)).apply(lambda x: round_string_float(x))
    
def multi_conversion(numbers):  
    try:
        if (numbers.find('"') > -1 or numbers.find('in') > -1) and numbers.find('ft') > -1:
            try:
                numbers=re.sub(r'ft\.','ft',str(numbers))
                numbers=re.sub(r'in\.','in',str(numbers))
                ft,inch=numbers.split('ft')
                ft=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(ft))
                inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(inch))
                ft_num=float(ft)*12
                total=ft_num+float(inch)
                return str(round_string_float(total))+' in'
            except:
                return str(numbers)
    except:
        return numbers
                       
    if numbers.find('yd') > -1:
        try:
            yd=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(numbers))
            new_yd=individual_value(yd)
#             new_yd=float(new_yd)*12*3
            return str(round_string_float(new_yd))+' yd'
        except:
            return numbers
        
    elif numbers.find('ft') > -1 or numbers.find("'") > -1 :
        try:
            ft=re.sub(r"yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\'|�|•|∩┐╜|\+|°|",'',str(numbers))
            new_ft=individual_value(ft)
#             new_ft=float(new_ft)*12
            return str(round_string_float(new_ft))+' ft'
        except:
            return numbers
        
    elif numbers.find('"') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' in'
        except:
            return numbers
    elif numbers.find('in') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' in'
        except:
            return numbers
        
    elif numbers.find('mm') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' mm'
        except:
            return numbers
        
    elif numbers.find('m') >-1 or numbers.find('M') >-1:
        try:
            inch=re.sub(r'yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' m'
        except:
            return numbers
    
    elif numbers.find('gal') >-1:
        try:
            inch=re.sub(r'gal\.|oz\.|yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' gal'
        except:
            return numbers
    elif numbers.find('oz') >-1:
        try:
            inch=re.sub(r'min\.|hr\.|gal\.|oz\.|yd\.?|in\.?|ft\.?|[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°|','',str(numbers))
            new_inch=individual_value(inch)
            new_inch=float(new_inch)
            return str(round_string_float(new_inch))+' oz'
        except:
            return numbers
    else:
        return numbers
    
def individual_value(one):
    new_one=round_string_float(re_extract(regex_year_month_day,(remove_frac(remove_fraction(date_to_decimal(re.sub(r'(?<=\d)\s?-',' ',str(remove_date_year_last(remove_date_year_first(remove_basic_date(re.sub(r'00:00:00|00:00|0:00','',str(re.sub(r'in\.|[a-z]|[A-z]|\"|�|•|∩┐╜|\+|°|,|lb.|in\.|Ã‚Â·|\(|\)','',str(one)))).rstrip())))))))))))
    return new_one
#########################################################################


# TEMP RANGES
#########################################################################
# Converts Above 40 Degrees F  and Above 0
def above_freezing(numbers): 
    if numbers.find('above freezing') > -1:   
        numbers=32
        return numbers
    elif numbers.find('above ') > -1:   
        numbers=re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(numbers))
        return numbers
    elif numbers.find('Greater ') > -1:   
        numbers=re.sub(r'[a-z]|[A-z]|\"|�|•| ','',str(numbers))
        return numbers
    else:
        return numbers
# data['min']=data['min'].apply(lambda x: above_freezing(x))
    
#Splits temp ranges: 60 to 100 Degrees F, -30 to 200 Degrees F, 50-100 Degrees
def split_height(data, col, dimension):
    data[col]=data[col].apply(lambda x:str(x).lower()) 
    data[['min_'+dimension, 'max_'+dimension]]=data[col].str.split('to',expand=True,)
    data['min_'+dimension]=data['min_'+dimension].apply(lambda x: above_freezing(x)).apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure)))
    data['max_'+dimension]=data['max_'+dimension].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°','',str(measure)))   
    
def split_temp(data, col, dimension):
    data[col]=data[col].apply(lambda x:x.lower())
    data[['min_'+dimension, 'max_'+dimension]]=data[col].str.split(' to |(?<=\d)\s?-|max|below|less(?=u)',expand=True,)
    data['min_'+dimension]=data['min_'+dimension].apply(lambda x: above_freezing(x)).apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•|°| ','',str(measure)))
    data['max_'+dimension]=data['max_'+dimension].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°','',str(measure)))

#########################################################################

def add_unit(df, col, unit):
    df[col] = df[col].fillna('').astype(str) + unit
    df.loc[df[col] == unit, col] = ''
    
def remove_metric(numbers, inputs):
    if str(numbers).find(inputs) > -1:
        numbers=''
        return numbers
    else:
        return numbers

In [140]:
Inputs=[
    'This is the first fraction 1/2',
]

def frac2string(s):
    i, f = s.groups(0)
    f = Fraction(f)
    return str(int(i) + float(f))

[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, s) for s in Inputs] 
[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x) for s in Inputs] 

['This is the first fraction 0.5']

In [ ]:
x=

['This is a fraction 0.5']

In [128]:
str(eval(x.group(0).replace(' ','+').replace('-',  '+'))), regex=True

SyntaxError: cannot assign to function call (<ipython-input-128-c8c72f083139>, line 1)

In [126]:
x=['10" to 25-1/2 in and 30.25 in']

In [122]:
from fractions import Fraction
y=1/2
Fraction(y)
float(y)

0.5

In [112]:
for s in str(x):
    sum(Fraction(x) for x in s)

ValueError: Invalid literal for Fraction: '['